In [1]:
import spacy
import pandas as pd
import networkx as nx
nlp = spacy.load("en_core_web_sm")
print("Environment ready")

Environment ready


In [2]:
from pathlib import Path
Path.cwd()

PosixPath('/Users/joenockels/douglass-kg/notebooks')

In [3]:
from pathlib import Path
import hashlib

# -----------------------------
# Define paths
# -----------------------------
PROJECT_ROOT = Path.cwd().parent  # <-- move up from notebooks
RAW_PATH = PROJECT_ROOT / "data" / "raw" / "HP.txt"
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "HP_clean.txt"

# -----------------------------
# Function to compute SHA256 checksum
# -----------------------------
def file_checksum(path: Path) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        h.update(f.read())
    return h.hexdigest()

# -----------------------------
# Load raw text
# -----------------------------
with open(RAW_PATH, "r", encoding="utf-8") as f:
    raw_text = f.read()

print(f"Loaded raw text: {len(raw_text)} characters")
print("SHA256 checksum:", file_checksum(RAW_PATH))

# -----------------------------
# Light normalization (non-destructive)
# -----------------------------
clean_text = raw_text.replace("\r\n", "\n").strip()
print("Cleaned text length:", len(clean_text))

# -----------------------------
# Save cleaned working copy
# -----------------------------
PROCESSED_PATH.parent.mkdir(parents=True, exist_ok=True)  # Make sure folder exists
with open(PROCESSED_PATH, "w", encoding="utf-8") as f:
    f.write(clean_text)

print(f"Clean working copy saved at {PROCESSED_PATH}")

Loaded raw text: 41624 characters
SHA256 checksum: f627b8559d8066503236268348a34f048b605cee7e63c577fd9dd0aee33cfae6
Cleaned text length: 41623
Clean working copy saved at /Users/joenockels/douglass-kg/data/processed/HP_clean.txt


In [4]:
# regex to separate HP entries based on dates, making a time_aware backdrop to the knowledge graph

import re

date_pattern = r'''
^(
    # 1️⃣ Full date: 14 September 1886
    \d{1,2}\s+
    (January|February|March|April|May|June|July|August|September|October|November|December)
    (?:\s+\d{4})?

    |

    # 2️⃣ Weekday, 16
    (Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),
    \s*\d{1,2}

    |

    # 3️⃣ 18 September (no year)
    \d{1,2}\s+
    (January|February|March|April|May|June|July|August|September|October|November|December)
)
'''

In [ ]:
# regex to separate DD entries based on dates, making a time_aware backdrop to the knowledge graph

import re

# Pattern explanation:
# (Jan|Feb|...|Dec) - matches abbreviated month names
# :? - optional colon
# \s* - optional whitespace
# \d{1,2} - one or two digits for day
# \.? - optional period at end
date_pattern = r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sept|Oct|Nov|Dec):?\s*\d{1,2}\.?'

In [11]:
# split HP based on date_pattern

matches = list(re.finditer(
    date_pattern,
    clean_text,
    flags=re.MULTILINE | re.VERBOSE | re.IGNORECASE
))

In [5]:
# split DD based on date_pattern

entries = re.split(date_pattern, clean_text)

# Find all matched dates to keep as metadata
dates = re.findall(date_pattern, clean_text)

print(f"Found {len(dates)} date headers")
print("First 5 dates:", dates[:5])
print("First entry snippet:", entries[1][:200])  # entries[0] may be preamble text

Found 0 date headers
First 5 dates: []


IndexError: list index out of range

In [14]:
# reassemble diary entries 

diary_entries = []

for i, match in enumerate(matches):
    start = match.end()
    end = matches[i + 1].start() if i + 1 < len(matches) else len(clean_text)

    diary_entries.append({
        "date": match.group(0).strip(),
        "text": clean_text[start:end].strip()
    })

print("First structured entry:")
print(diary_entries[0])

First structured entry:
{'date': '14 September 1886', 'text': 'City of Rome.\nDrove to the pier in a hansom, from Grand Central Hotel, at 1 o’clock, a\nbeautiful day, and after looking up our luggage went on board, and remained there,\nlooking over the ship, watching the loading of the vessel and visiting with friends. In the\nevening Frederick went on deck, but I was tired and undressed and mounted my perch,\nthe upper berth, and though we had pulled close on the dock and were all night receiving\nand loading freight just outside our port hole, I slept all night. As I said we went on board\nearly, eager to see our room. We found it a very comfortable little place, the space\nopposite the berths and under the window a port hole, a round eye of glass, is occupied\nby the couch. Over it and over each berth is a wall pocket and over the stationary wash\nbowl, between it and the mirror a fresh linen pocket for brushes, fresh water bottle,\npneumatic bell call, and altogether a nice cosy li

In [16]:
# covert to a standardised timeset YYYY-MM-DDT00:00:00 format, needed for named entity recognition and the eventual KG in Gephi using timestamps

from datetime import datetime
import re

def parse_diary_date(date_str, current_year, last_month=None):

    original = date_str
    clean = date_str.lower()
    clean = re.sub(r'[.,:]', '', clean)
    clean = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', clean)
    clean = clean.strip()

    parts = clean.split()

    day = None
    month = None
    year = current_year

    # Case 1: 14 September 1886
    if len(parts) >= 2 and parts[0].isdigit() and parts[1] in month_map:
        day = int(parts[0])
        month = month_map[parts[1]]
        if len(parts) == 3 and parts[2].isdigit():
            year = int(parts[2])

    # Case 2: 18 September
    elif len(parts) == 2 and parts[0].isdigit() and parts[1] in month_map:
        day = int(parts[0])
        month = month_map[parts[1]]

    # Case 3: Thursday 16
    elif parts[0] in weekdays and len(parts) == 2 and parts[1].isdigit():
        if last_month is None:
            raise ValueError(f"No month context for weekday entry: {original}")
        day = int(parts[1])
        month = last_month

    else:
        raise ValueError(f"Unrecognised date format: {original}")

    # Detect rollover ONLY if month explicitly changes
    if last_month and month and month < last_month:
        year += 1

    # 🔎 Validate date safely
    try:
        dt = datetime(year, month, day)
    except ValueError as e:
        raise ValueError(
            f"Invalid date constructed from '{original}' → "
            f"year={year}, month={month}, day={day}"
        ) from e

    iso = dt.strftime("%Y-%m-%dT00:00:00")
    return iso, year, month

In [ ]:
# automatically rolls over for DD, which stretches 188 - 1887, with the entries being in chronologically order 
current_year = 1886  # start year
standardized_dates = []

for entry in diary_entries:
    std_date, current_year = parse_diary_date_with_year(entry["date"], current_year)
    entry["standard_date"] = std_date
    standardized_dates.append(std_date)

# Quick check
for e in diary_entries[:10]:
    print(e["date"], "→", e["standard_date"])

In [18]:
# at this point, the text entry is preserved, with ISO standard dates 

import json

structured_path = PROJECT_ROOT / "data" / "processed" / "HP_diary_entries.json"

with open(structured_path, "w", encoding="utf-8") as f:
    json.dump(diary_entries, f, indent=2)

print(f"Structured diary saved to {structured_path}")

Structured diary saved to /Users/joenockels/douglass-kg/data/processed/HP_diary_entries.json


In [19]:
import csv

csv_path = PROJECT_ROOT / "data" / "processed" / "HP_diary_entries.csv"

with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=["date", "iso_date", "text"]
    )
    writer.writeheader()
    writer.writerows(diary_entries)

print(f"CSV diary saved to {csv_path}")


CSV diary saved to /Users/joenockels/douglass-kg/data/processed/HP_diary_entries.csv
